In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim import corpora, models, similarities
import re
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize

In [2]:
# Đọc dữ liệu
hotel_comments_df = pd.read_csv('data/hotel_comments_cleaned.csv')
hotel_info_df = pd.read_csv('data/hotel_info_cleaned.csv')

In [3]:
# Đọc danh sách từ dừng từ file
STOP_WORD_FILE = 'files/vietnamese-stopwords.txt'
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')

In [4]:
# Sử dụng mô tả khách sạn để làm Content-Based Filtering
descriptions = hotel_info_df['Hotel_Description'].fillna('')

In [5]:
# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Xóa các ký tự đặc biệt và xuống dòng
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[0-9]+', '', text)  # Xóa các ký tự số

    # Tách văn bản thành các câu
    sentences = sent_tokenize(text)

    # Xử lý từng câu
    processed_text = []
    for sentence in sentences:
        # Tách từ và gán nhãn từ loại
        words_pos = pos_tag(word_tokenize(sentence, format='text'))

        # Chỉ lấy những từ không phải từ dừng và là danh từ, động từ hoặc tính từ
        filtered_words = [word.lower() for word, pos in words_pos if word.lower() not in stop_words and pos in ['N', 'V', 'A'] and word not in ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&']]
        
        # Thêm vào danh sách đã xử lý
        processed_text.extend(filtered_words)

    return processed_text

In [6]:
# Tiền xử lý mô tả khách sạn
texts = descriptions.apply(preprocess_text).tolist()

In [7]:
texts[:1]

[['lưu_trú',
  'tuyệt_vời',
  'nằm',
  'trung_tâm',
  'thành_phố',
  'đẳng_cấp',
  'thuận_lợi',
  'trung_tâm',
  'sân_bay',
  'phút',
  'đi',
  'xe_khách_sạn',
  'xây_dựng',
  'hiện',
  'tổng_cộng',
  'số_lượng',
  'đáp_ứng',
  'nhu_cầu lưu_trú',
  'du_khách',
  'thoải_mái',
  'tiện_nghi',
  'cho_phép',
  'du_khách',
  'thư_giãn',
  'tận_hưởng',
  'tiện_ích',
  'dịch_vụ tuyệt_vời',
  'cho_phép',
  'lưu_trú',
  'miễn_phí',
  'điều_kiện thuận_lợi',
  'trẻ',
  'tọa_lạc',
  'du_khách',
  'tiện_nghi',
  'giải_trí',
  'tuyệt_vời',
  'cửa_hàng',
  'đa_dạng',
  'đáp_ứng',
  'nhu_cầu mua_sắm',
  'thử_vận',
  'may',
  'ghé',
  'thăm',
  'sòng',
  'bạc',
  'trải_nghiệm',
  'trò_chơi',
  'độc_đáo',
  'quầy',
  'thân_thiện',
  'thưởng_thức',
  'đồ',
  'uống',
  'thơm',
  'ngon',
  'thư_giãn',
  'dạo',
  'làm_đẹp',
  'chăm_sóc sức',
  'khỏe',
  'ghé',
  'thăm',
  'spa',
  'tận_hưởng',
  'liệu_pháp',
  'massage',
  'thư_giãn',
  'ngâm',
  'bồn',
  'nóng thư_giãn',
  'xông_hơi',
  'tận_hưởng',
  'liệu

In [8]:
# Tạo một dictionary cho các tài liệu
dictionary = corpora.Dictionary(texts)

In [9]:
dictionary.token2id

{'agoda cam_kết': 0,
 'agoda du_khách': 1,
 'an_toàn': 2,
 'bao_gồm': 3,
 'bay': 4,
 'biển': 5,
 'buffet': 6,
 'bài_tập': 7,
 'bánh': 8,
 'bãi': 9,
 'bóng_chuyền': 10,
 'bơi': 11,
 'bơi ngoài_trời': 12,
 'bạc': 13,
 'bể': 14,
 'bỏ_qua': 15,
 'bồn': 16,
 'bữa': 17,
 'cho_phép': 18,
 'chuyên_nghiệp': 19,
 'chuyến': 20,
 'chân': 21,
 'chùa': 22,
 'chú_ý': 23,
 'chăm_sóc': 24,
 'chăm_sóc sức': 25,
 'chăm_sóc tận_tâm': 26,
 'chất_lượng': 27,
 'chỗ': 28,
 'chợ': 29,
 'cuối_cùng khách_sạn': 30,
 'cuộc_sống': 31,
 'cà_phê': 32,
 'cát': 33,
 'công_cộng': 34,
 'công_sức': 35,
 'công_viên': 36,
 'căng_thẳng': 37,
 'cơ_sở vật_chất': 38,
 'cảnh_quan': 39,
 'cửa_hàng': 40,
 'da cá_sấu': 41,
 'di_chuyển': 42,
 'diện_tích': 43,
 'diện_tích mét_vuông': 44,
 'du_khách': 45,
 'du_lịch': 46,
 'duy_trì': 47,
 'dạo': 48,
 'dễ_dàng': 49,
 'dịch_vụ': 50,
 'dịch_vụ chuyên_nghiệp': 51,
 'dịch_vụ dọn': 52,
 'dịch_vụ tuyệt_vời': 53,
 'dịp': 54,
 'dọn': 55,
 'dừng': 56,
 'ghé': 57,
 'giao': 58,
 'giá': 59,
 'giá_c

In [25]:
# Lưu lại dictionary ra file
print(type(dictionary))

<class 'gensim.corpora.dictionary.Dictionary'>


In [27]:
dictionary.save_as_text("dictionary.txt")

In [28]:
dictionary.save('dictionary.dict')

In [10]:
# Số lượng từ đặc trưng trong từ điển
feature_cnt = len(dictionary.token2id)

In [11]:
feature_cnt

9366

In [12]:
# Chuyển đổi tài liệu sang định dạng bag-of-words (BoW)
corpus = [dictionary.doc2bow(text) for text in texts]

In [13]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 6),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 5),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 2),
 (19, 2),
 (20, 2),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 4),
 (33, 1),
 (34, 3),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 4),
 (41, 1),
 (42, 3),
 (43, 1),
 (44, 5),
 (45, 28),
 (46, 6),
 (47, 1),
 (48, 2),
 (49, 8),
 (50, 22),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 6),
 (58, 1),
 (59, 3),
 (60, 1),
 (61, 1),
 (62, 10),
 (63, 6),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 3),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 3),
 (77, 2),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 2),
 (84, 1),
 (85, 9),
 (86, 1),
 (87, 9),
 (88, 1),
 (89, 8),
 (90, 1),
 (91, 1),
 (92, 2),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 3),
 (99, 1),
 (100, 

In [14]:
# # Sử dụng mô hình TF-IDF để xử lý corpus
# tfidf = models.TfidfModel(corpus)

# Huấn luyện mô hình LSI
lsi_model = models.LsiModel(corpus, id2word=dictionary, num_topics=100)

In [15]:
# # Tính toán sự tương tự trong ma trận thưa
# index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)

# Tạo một chỉ số tương đồng
index = similarities.MatrixSimilarity(lsi_model[corpus])

In [16]:
df_1 = pd.DataFrame(index)
df_1

,0,1,2,3,4,5,6,7,8,9,...,730,731,732,733,734,735,736,737,738,739
0,1.000000,0.889916,0.901191,0.836633,0.873696,0.819981,0.915786,0.881765,0.843326,0.395833,...,0.338691,0.840059,0.336934,0.876655,0.337841,0.224488,0.280425,0.180452,0.387880,0.861221
1,0.889916,1.000000,0.907345,0.859154,0.889439,0.861618,0.912890,0.870421,0.864198,0.378553,...,0.310958,0.889779,0.295420,0.866035,0.309300,0.218421,0.284117,0.215508,0.368152,0.848550
2,0.901191,0.907345,1.000000,0.863605,0.911250,0.854755,0.905562,0.902465,0.896331,0.405965,...,0.323596,0.873315,0.310151,0.914448,0.342337,0.231759,0.275829,0.235503,0.427898,0.863589
3,0.836633,0.859154,0.863605,1.000000,0.853594,0.890726,0.831861,0.849048,0.870677,0.354342,...,0.259296,0.860343,0.286733,0.838747,0.267745,0.237302,0.208908,0.235242,0.371342,0.847143
4,0.873696,0.889439,0.911250,0.853594,1.000000,0.853527,0.884577,0.862329,0.879114,0.394108,...,0.330595,0.836911,0.333159,0.848709,0.334218,0.269580,0.309222,0.275097,0.441430,0.847306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0.224488,0.218421,0.231759,0.237302,0.269580,0.200072,0.194591,0.253755,0.220860,0.361047,...,0.744042,0.227293,0.400367,0.270592,0.339748,1.000000,0.427024,0.438497,0.418142,0.258282
736,0.280425,0.284117,0.275829,0.208908,0.309222,0.235019,0.299242,0.266503,0.242570,0.668137,...,0.550580,0.267054,0.737502,0.263998,0.787002,0.427024,1.000000,0.599864,0.816591,0.240969
737,0.180452,0.215508,0.235503,0.235242,0.275097,0.233235,0.213146,0.218689,0.280531,0.545916,...,0.435241,0.280800,0.577302,0.229224,0.584036,0.438497,0.599864,1.000000,0.662251,0.271143
738,0.387880,0.368152,0.427898,0.371342,0.441430,0.350761,0.405663,0.404503,0.379614,0.743825,...,0.609766,0.360852,0.777499,0.413045,0.802081,0.418142,0.816591,0.662251,1.000000,0.376311


In [17]:
# # Hàm tìm các khách sạn tương tự sử dụng mô hình TF-IDF
# def get_similar_hotels_tfidf(hotel_id, index=index, top_n=5):
#     # Lấy chỉ số của khách sạn
#     idx = hotel_indices[hotel_id]
    
#     # Chuyển đổi mô tả khách sạn sang không gian TF-IDF
#     vec_bow = dictionary.doc2bow(preprocess_text(descriptions[idx]))
#     vec_tfidf = tfidf[vec_bow]
    
#     # Tính toán độ tương đồng
#     sim_scores = index[vec_tfidf]
    
#     # Sắp xếp các khách sạn dựa trên điểm tương đồng
#     sim_scores = sorted(enumerate(sim_scores), key=lambda item: -item[1])
    
#     # Lấy điểm số của top n khách sạn tương tự nhất
#     sim_scores = sim_scores[1:top_n+1]
    
#     # Lấy chỉ số của các khách sạn
#     hotel_indices_similar = [i[0] for i in sim_scores]
    
#     # Trả về danh sách các khách sạn tương tự nhất
#     return hotel_info_df['Hotel_Name'].iloc[hotel_indices_similar]

# Hàm tìm các khách sạn tương tự sử dụng mô hình LSI
def get_similar_hotels_gensim(hotel_id, lsi_model=lsi_model, index=index, top_n=5):
    # Lấy chỉ số của khách sạn
    idx = hotel_indices[hotel_id]
    
    # Chuyển đổi mô tả khách sạn sang không gian LSI
    vec_bow = dictionary.doc2bow(preprocess_text(descriptions[idx]))
    vec_lsi = lsi_model[vec_bow]
    
    # Tính toán độ tương đồng
    sim_scores = index[vec_lsi]
    
    # Sắp xếp các khách sạn dựa trên điểm tương đồng
    sim_scores = sorted(enumerate(sim_scores), key=lambda item: -item[1])
    
    # Lấy điểm số của top n khách sạn tương tự nhất
    sim_scores = sim_scores[1:top_n+1]
    
    # Lấy chỉ số của các khách sạn
    hotel_indices_similar = [i[0] for i in sim_scores]
    
    # Trả về danh sách các khách sạn tương tự nhất
    return hotel_info_df['Hotel_Name'].iloc[hotel_indices_similar]

In [18]:
# Vector hóa TF-IDF để tính độ tương đồng cosine
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)

In [19]:
# Tính ma trận độ tương đồng cosine
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,...,730,731,732,733,734,735,736,737,738,739
0,1.000000,0.919637,0.912405,0.906813,0.921963,0.900257,0.910453,0.894159,0.892167,0.676761,...,0.590264,0.870460,0.616913,0.825925,0.628891,0.452473,0.295029,0.392339,0.630069,0.874332
1,0.919637,1.000000,0.931005,0.913594,0.927556,0.915264,0.919998,0.902335,0.913164,0.677992,...,0.592291,0.903261,0.601615,0.833140,0.632853,0.484745,0.308885,0.404979,0.644019,0.881485
2,0.912405,0.931005,1.000000,0.916239,0.934884,0.922665,0.904921,0.900732,0.906019,0.666070,...,0.587460,0.900748,0.600497,0.840867,0.628797,0.473723,0.297140,0.397192,0.643391,0.875815
3,0.906813,0.913594,0.916239,1.000000,0.914462,0.921026,0.889093,0.875146,0.892747,0.642501,...,0.571081,0.880097,0.585859,0.819561,0.600816,0.462736,0.294324,0.401312,0.614463,0.878299
4,0.921963,0.927556,0.934884,0.914462,1.000000,0.910535,0.907572,0.899432,0.902525,0.679875,...,0.597708,0.881415,0.621772,0.827758,0.631632,0.488257,0.310345,0.415947,0.664918,0.869397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0.452473,0.484745,0.473723,0.462736,0.488257,0.448721,0.458153,0.483224,0.472078,0.588267,...,0.681117,0.443158,0.585531,0.456166,0.586803,1.000000,0.540166,0.419183,0.590233,0.458101
736,0.295029,0.308885,0.297140,0.294324,0.310345,0.286084,0.309212,0.301946,0.295624,0.390690,...,0.384948,0.295331,0.395675,0.311057,0.402081,0.540166,1.000000,0.314556,0.432573,0.303822
737,0.392339,0.404979,0.397192,0.401312,0.415947,0.398205,0.402418,0.396661,0.398006,0.515457,...,0.454711,0.398991,0.540670,0.372006,0.521824,0.419183,0.314556,1.000000,0.536962,0.402275
738,0.630069,0.644019,0.643391,0.614463,0.664918,0.612573,0.631987,0.647753,0.636404,0.776265,...,0.664924,0.623105,0.777764,0.622035,0.748171,0.590233,0.432573,0.536962,1.000000,0.618532


In [21]:
# Tạo một ánh xạ từ tên khách sạn đến chỉ số
hotel_indices = pd.Series(hotel_info_df.index, index=hotel_info_df['Hotel_ID']).drop_duplicates()

In [22]:
# Hàm lấy các khách sạn tương tự sử dụng độ tương đồng cosine
def get_similar_hotels_cosine(hotel_id, cosine_sim=cosine_sim, top_n=5):
    # Lấy chỉ số của khách sạn
    idx = hotel_indices[hotel_id]
    
    # Lấy điểm tương đồng cặp
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sắp xếp các khách sạn dựa trên điểm tương đồng
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Lấy điểm số của top n khách sạn tương tự nhất
    sim_scores = sim_scores[1:top_n+1]
    
    # Lấy chỉ số của các khách sạn
    hotel_indices_similar = [i[0] for i in sim_scores]
    
    # Trả về danh sách các khách sạn tương tự nhất
    return hotel_info_df['Hotel_Name'].iloc[hotel_indices_similar]

In [23]:
# Người dùng nhập Hotel ID
hotel_id_input = input("Nhập Hotel ID: ")

# Sử dụng Hotel ID để lấy tên khách sạn
if hotel_id_input in hotel_indices:
    similar_hotels_cosine = get_similar_hotels_cosine(hotel_id_input)
    similar_hotels_gensim = get_similar_hotels_gensim(hotel_id_input)

    print("Khách sạn tương tự sử dụng Cosine Similarity:")
    print(similar_hotels_cosine)
    print('+' * 56)
    print("Khách sạn tương tự sử dụng Gensim LSI:")
    print(similar_hotels_gensim)
else:
    print("Hotel ID không tồn tại.")

Khách sạn tương tự sử dụng Cosine Similarity:
37                                 Dubai Nha Trang Hotel
49             Khách sạn V Nha Trang (V Hotel Nha Trang)
208                        Khách Sạn Vesna (Vesna Hotel)
226                        Khách sạn Sochi (Sochi Hotel)
30     Khách sạn New Sun Nha Trang (New Sun Hotel Nha...
Name: Hotel_Name, dtype: object
++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Khách sạn tương tự sử dụng Gensim LSI:
226                        Khách sạn Sochi (Sochi Hotel)
364                              Angella Hotel Nha Trang
625    Khách sạn LeMore Nha Trang (LeMore Hotel Nha T...
6                Khách sạn Mega Light (Mega Light Hotel)
95                     Khách sạn Alibaba (Alibaba Hotel)
Name: Hotel_Name, dtype: object


##### Hiệu quả của mô hình:
- Cosine Similarity: Mô hình này tốt trong việc cung cấp khuyến nghị dựa trên sự tương đồng về nội dung mô tả trực tiếp. Nó thích hợp cho việc gợi ý dựa trên các từ khóa cụ thể và là một phương pháp khá đơn giản và hiệu quả.
- Gensim LSI: Mô hình này thể hiện khả năng khám phá sâu hơn vào dữ liệu, cung cấp các khuyến nghị đa dạng và phức tạp hơn. Nó thích hợp cho các ứng dụng cần nhận diện sự liên kết ngữ nghĩa phức tạp giữa các tài liệu.

##### Khuyến nghị sử dụng:
- Đối với những người dùng tìm kiếm khuyến nghị nhanh chóng và dựa trên từ khóa cụ thể, Cosine Similarity có thể là lựa chọn tốt.
- Đối với các ứng dụng yêu cầu độ sâu và đa dạng về nội dung, Gensim LSI sẽ phù hợp hơn.

## Lưu mô hình

In [24]:
# Save cosine_sim to file
import pickle
with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)



with open('gensim_lsi.pkl', 'wb') as f:
    pickle.dump(lsi_model, f)